In [81]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [82]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.float)

print(bits_info)

tensor([[1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 1., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 1.],
        [1., 1., 0., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 1., 1.],
        [0., 1., 1., 0.],
        [0., 1., 0., 1.],
        [1., 0., 1., 1.],
        [0., 1., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [1., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [0., 0., 1., 0.],
        [0.,

Hamming(7,4) Encoder

In [83]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.float)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [84]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 1., 0.],
        [1., 0., 1., 0., 1., 0., 1.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1., 0., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 0., 0., 1.],
        [1.,

BPSK Modulator + Noise

In [85]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        data: data received from the Hamming(7,4) encoder(Tensor)
        symbol_rate: Symbol rate in Hz
        carrier_freq: Carrier frequency in Hz
        snr_dB: Signal-to-noise ratio in dB

    Returns:
        time:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, data, symbol_rate, carrier_freq, snr_dB):
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
            # Time vector
            time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
        
            # Generate carrier signal
            carrier = torch.cos(2 * torch.pi * carrier_freq * time)
        
            # Modulate the signal
            modulated_signal = bits * carrier
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(-snr_dB / 10))
            noise = torch.sqrt(noise_power) * torch.randn(len(modulated_signal))
            noised_signal = modulated_signal + noise
            data[i] = noised_signal
    
       
        return time, data

In [86]:
data = encoded_codeword  # Binary data
symbol_rate = 15  # Symbol rate in Hz
carrier_freq = 100  # Carrier frequency in Hz
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
time, modulated_noise_signal = modulator(data, symbol_rate, carrier_freq, snr_dB)
print(modulated_noise_signal)

tensor([[ 0.7453, -0.4620, -0.2500,  1.1472, -0.6383, -0.5421,  1.0159],
        [-0.9008,  0.5603,  0.3983,  0.6971, -0.2569, -0.5776,  1.1273],
        [ 0.8734, -0.8381,  0.2871,  0.7665, -0.6254,  0.3445, -1.1540],
        [-1.1446,  0.2309, -0.5542, -1.0126,  0.5343, -0.5535,  0.9962],
        [-1.2062, -0.7443,  0.4220,  1.0387,  0.5108, -0.5050, -0.9965],
        [ 1.0912, -0.5700,  0.6913, -1.3200,  0.6654, -0.5304,  0.8996],
        [-1.1356, -0.4935,  0.2297,  0.8851,  0.6132, -0.2982, -1.0169],
        [ 0.6933,  0.7293,  0.7693, -1.2303, -0.5011, -0.7560, -0.6969],
        [ 1.2241,  0.4316, -0.5059, -0.9118, -0.1388,  0.4279,  0.7819],
        [ 0.9793, -0.1205,  0.6057, -0.8664,  0.4432, -0.4690,  0.8753],
        [-0.8919,  0.5296,  0.4992, -1.0284,  0.3750,  0.4294, -1.2095],
        [ 0.7453,  0.3750, -0.5425,  0.6915,  0.4455, -0.6907, -1.1203],
        [-0.9582, -0.3891, -0.6088, -1.5121, -0.4374, -0.6417, -1.1027],
        [-1.1741, -0.3791,  0.5753,  1.1954,  0.614

LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [103]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal.
        noise_std (float): Standard deviation of the noise.

    Returns:
        torch.Tensor: Log Likelihood Ratio (LLR) values.
    """
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))
    # likelihood_0 = 2 * (signal - 1) / noise_std**2
    # likelihood_1 = 2 * (signal + 1) / noise_std**2

    # Calculate the LLR
    # llr_values = likelihood_0 / likelihood_1
    llr = 2 * signal / noise_std**2

    # return llr_values, llr
    return llr


In [104]:
received_signal = torch.tensor([0.7453, -0.4620, -0.2500,  1.1472, -0.6383, -0.5421,  1.0159])
snr_dB = 15

llr = llr(received_signal, snr_dB)
print("LLR values2:", llr)

LLR values2: tensor([ 0.0471, -0.0292, -0.0158,  0.0726, -0.0404, -0.0343,  0.0643])


LDPC Decoder

In [96]:
import numpy as np

def ldpc_decode(received_codeword, max_iterations=50):
    # LDPC Parity-check matrix (7,4) example
    H = np.array([
        [1, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 1, 0]
    ])

    # LDPC Generator matrix (7,4) example
    G = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],
        [1, 0, 1, 1],
        [1, 1, 0, 1]
    ])

    # Initialize the received codeword and the decoded message
    y = np.array(list(map(int, received_codeword)))
    x_hat = np.zeros(G.shape[1])

    for iteration in range(max_iterations):
        # Syndrome check
        syndrome = np.mod(np.dot(H, y), 2)

        if np.all(syndrome == 0):
            # If the syndrome is all zeros, the decoding is successful
            break

        # Compute the log-likelihood ratios (LLRs) using the sum-product algorithm
        llrs = np.zeros(G.shape[1])
        for j in range(G.shape[1]):
            connected_checks = np.nonzero(H[:, j])[0]
            llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))

        # Make hard decisions based on LLRs
        x_hat = np.where(llrs > 0, 1, 0)

        # Update the received codeword
        y = np.mod(np.dot(G, x_hat) + y, 2)

    decoded_message = ''.join(map(str, x_hat.astype(int)))
    return decoded_message

def main():
    # Example usage
    received_codeword = "1101001"  # Received codeword with possible errors
    decoded_message = ldpc_decode(received_codeword)

    print(f"Received codeword: {received_codeword}")
    print(f"Decoded message: {decoded_message}")

if __name__ == "__main__":
    main()


Received codeword: 1101001
Decoded message: 1101


/var/folders/71/hry96qw555j_rlvg3tfgrh180000gn/T/ipykernel_29804/2802857671.py:38: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))
